# Orthogonal Decomposition of Antisymmetric Matrices

Generalized implementation of method described in:

[1]: Hanson, Jason. "Rotations in three, four, and five dimensions." arXiv preprint arXiv:1103.5263 (2011).

### Problem Statement

Given $f$ be a real $n \times n$ antisymmetric matrix, find the orthogonal decomposition $f = \sum _{j}f_{j}$ where $j = 1, \dots, m$ and $m = \lfloor \frac{n}{2} \rfloor$.


### Motivation

If we are able to write an antisymmetric matix $f$ as a the sum of wedge products $f_j$ whose two–planes are mutually orthogonal, then:

$(\sum_{j}f_{j})^{k}=\sum_{j}f_{j}^{k}$

We can use this to compute the matrix exponential. From Taylor series expansion:

$R = exp(f) = exp(\sum_{j}f_{j}) = I + \sum_{j}(exp(f_{j})-I)$

Where:

$exp(f_{j}) = exp(\theta (\mathbf{u} \wedge \mathbf{v})) = I - (1-cos(\theta)\operatorname{proj}_{\mathbf{uv}} + sin(\theta)(\mathbf{u} \wedge \mathbf{v})$

## Preliminaries

In [1]:
import types

import functools as ft
import itertools as it

import numpy as np
import sympy as sp

Basic operations and properties.

#### Outer Product

$\mathbf{u} \otimes \mathbf{v} = \mathbf{u} \mathbf{v}^{\textsf{T}}$

#### Wedge Product

$\mathbf{u} \wedge \mathbf{v} = \mathbf{u} \otimes \mathbf{v} - \mathbf{v} \otimes \mathbf{u}$

#### Properties

For all vectors $\mathbf{u}, \mathbf{v}$ we have (see [1], Lemma 3.1.):

- $\operatorname{tr}(\mathbf{u} \otimes \mathbf{v}) = \mathbf{u} \cdot \mathbf{v}$
- $(\mathbf{u} \wedge \mathbf{v})^2 = -|\mathbf{u} \wedge \mathbf{v}|^2 \operatorname{proj}_{\mathbf{uv}}$
- $(\mathbf{u} \wedge \mathbf{v})^3 = -|\mathbf{u} \wedge \mathbf{v}|^2 (\mathbf{u} \wedge \mathbf{v})$

Where $|\mathbf{u} \wedge \mathbf{v}|^2 = |\mathbf{u}|^2|\mathbf{v}|^2 - (\mathbf{u} \cdot \mathbf{v})^2$ and $\operatorname{proj}_{\mathbf{uv}}$ is orthogonal projection onto the plane spanned by $\mathbf{u}$ and $\mathbf{v}$.

In [2]:
def outer(u, v):
    return sp.Matrix([[u[i]*v[j] for j in range(len(v))] for i in range(len(u))])

def wedge(u, v):
    return outer(u,v) - outer(v,u)

def trace(a):
    return sp.simplify(sp.Trace(a))
    
def inner(a, b):
    """For symmetric or antisymmetric matrices."""
    if (isinstance(a, sp.Matrix) and isinstance(b, sp.Matrix)):
        return trace(a @ b.T)/2
    else:
        return np.trace(a @ b.T)/2

def norm(m):
    return sp.sqrt(inner(m, m))

def proj(m):
    return -(m @ m)/inner(m, m)

Optional: Define some geometries for testing.

In [3]:
def unit(n, dtype=np.float64):
    def _unit(i):
        result = np.zeros((n,), dtype=dtype)
        result[i] = 1
        return result
    return _unit

def geometry(n):
    result = types.SimpleNamespace()
    result.n = n
    result.basis = [unit(n)(i) for i in range(n)]
    for i, in it.combinations(range(n), 1):
        v = result.basis[i]
        setattr(result, "e{}".format(i), v)
    for i, j in it.combinations(range(n), 2):
        v = wedge(result.basis[i], result.basis[j])
        setattr(result, "e{}{}".format(i,j), v)
    return result

g2 = geometry(2)
g3 = geometry(3)
g4 = geometry(4)
g5 = geometry(5)
g6 = geometry(6)

Optional: Convenience methods for declaring symbols.

In [4]:
def vec(*args):
    return sp.Matrix([args])
def sym(s, n, **kwargs):
    r = sp.symbols(" ".join([f"{s}_{i}" for i in range(n)]), **kwargs)
    return [r] if n==1 else r
def symvec(s, n, **kwargs):
    return vec(*sym(s, n, **kwargs))

In [5]:
thetas = symvec(r"\theta", 9, positive = True, real = True)
f = sp.symbols(r"f")
x = sp.symbols(r"x")
y = sp.symbols(r"y")
fs = symvec("f", 9)
xs = symvec("x", 9)
ys = symvec("y", 9)

Method for eliminating variables from systems of polynomial equations.
(see https://github.com/sympy/sympy/pull/11485#issuecomment-238145994)

In [6]:
from sympy.polys.orderings import _ItemGetter, build_product_order

_ItemGetter.__hash__  = lambda self: hash(self.seq) 

def eliminate(eqs, vs):
    '''Method for eliminating variables from systems of polynomial equations'''
    O = build_product_order((('lex', v) for v in reversed(vs)), vs)
    G = sp.groebner(eqs, *vs, order=O)
    return G.exprs[-1]

### Orthogonal Decomposition of Antisymmetric Matrix (d=2,3)

$f = f_0$

(Trivial case.)

In [7]:
eqs = [sp.Eq(-2*(xs[0]**1), ys[0]),]

p = eliminate(eqs, xs)
p

2*x_0 + y_0

In [8]:
sp.Poly(p, xs[0]).coeffs()

[2, y_0]

In [9]:
def find_theta2s(f):
    f1 = f
    f2 = f1 @ f1
    y0 = np.trace(f2)
    p = np.poly1d([2, y0])
    return p.roots # -y0/2

In [10]:
np.sqrt(find_theta2s(g2.e01*123))

array([123.])

### Orthogonal Decomposition of Antisymmetric Matrix (d=4,5)

$f = f_0 + f_1$

#### Step 1

Find $\theta_0, \theta_1$, where $\theta_{i}^2 = |f_{j}|^2$.

(From lemma 3.1, $f_{i}^2 = -|f_{i}|^2*\operatorname{proj}_{i}$)

$f^2 = -|f_0|^2*\operatorname{proj}_0 -|f_1|^2*\operatorname{proj}_1$

$f^4 = |f_0|^4*\operatorname{proj}_0 + |f_1|^4*\operatorname{proj}_1$


(From lemma 3.1, $\operatorname{tr} (\operatorname{proj}_{uv}) = 2$)

$-\frac{1}{2}\operatorname{tr} (f^2) = |f_0|^2 + |f_1|^2$

$\frac{1}{2}\operatorname{tr} (f^4) = |f_0|^4 + |f_1|^4$

In [11]:
eqs = [
    sp.Eq(-2*(xs[0]**1 + xs[1]**1), ys[0]),
    sp.Eq(+2*(xs[0]**2 + xs[1]**2), ys[1])
]

p = eliminate(eqs, xs)
p

8*x_0**2 + 4*x_0*y_0 + y_0**2 - 2*y_1

In [12]:
sp.Poly(p, xs[0]).coeffs()

[8, 4*y_0, y_0**2 - 2*y_1]

In [13]:
def find_theta2s(f):
    f1 = f
    f2 = f1 @ f1
    f4 = f2 @ f2
    y0 = np.trace(f2)
    y1 = np.trace(f4)
    p = np.poly1d([8, 4*y0, y0**2 - 2*y1])
    return p.roots

In [14]:
_theta0 = 1
_theta1 = 2

_f = np.array(_theta0 * g4.e01 + _theta1 * g4.e23).astype(np.float32)

__theta0, __theta1 = np.sqrt(find_theta2s(_f))

_f, __theta0, __theta1

(array([[ 0.,  1.,  0.,  0.],
        [-1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  2.],
        [ 0.,  0., -2.,  0.]], dtype=float32),
 2.0,
 1.0)

#### Step 2

Express $f_0, f_1$ in terms of $f$ and $\theta_0, \theta_1$ by solving the following system (see [1] for derivation):

${\begin{bmatrix}1&1\\\theta_0^2&\theta_1^2\\\end{bmatrix}}\,{\begin{bmatrix}f_0\\f_1\\\end{bmatrix}}={\begin{bmatrix}f\\-f^3\\\end{bmatrix}}$

In [15]:
T = sp.Matrix([
    [thetas[0]**0,thetas[1]**0],
    [thetas[0]**2,thetas[1]**2]
])
_f = sp.Matrix([[f, -f**3]])
_fs = sp.Matrix([fs[:2]])
eq0 = sp.Eq(T @ _fs.T, _f.T)
eq0

Eq(Matrix([
[                        f_0 + f_1],
[\theta_0**2*f_0 + \theta_1**2*f_1]]), Matrix([
[    f],
[-f**3]]))

Solve for $f_0, f_1$.

*Careful!*

The cases where det(T) == 0 require special handling. Compute determinant separately and check.

$T^{-1} = \frac{1}{det(T)} adj(T)$

In [16]:
eq1 = sp.Eq(_fs.T, 1/T.det() * (T.adjugate() @ _f.T))
sp.simplify(eq1)

Eq(Matrix([
[f_0],
[f_1]]), Matrix([
[-f*(\theta_1**2 + f**2)/(\theta_0**2 - \theta_1**2)],
[ f*(\theta_0**2 + f**2)/(\theta_0**2 - \theta_1**2)]]))

In [17]:
def decompose_antisymmetric(f):
    f1 = f
    f2 = f @ f1
    f3 = f @ f2
    f4 = f @ f3
    y0 = np.trace(f2)
    y1 = np.trace(f4)
    t = np.poly1d([8, 4*y0, y0**2 - 2*y1]).roots
    dt = t[1] - t[0]
    if np.isclose(dt, 0):
        raise NotImplementedError
    f0 = (+f3 + t[1] * f1)/dt
    f1 = (-f3 - t[0] * f1)/dt
    return f0, f1

In [18]:
def is_ortho(x, y):
    return np.isclose(inner(x, y), 0)

In [19]:
_theta0 = 1
_theta1 = 2

_f = np.array(_theta0 * g4.e01 + _theta1 * g4.e23).astype(np.float32)

_f0, _f1 = decompose_antisymmetric(_f)

#print(_f, _f0, _f1)

np.allclose(_f, _f0 + _f1), is_ortho(_f0, _f1)

(True, True)

### Orthogonal Decomposition of Antisymmetric Matrix (d=6,7)

$f = f_0 + f_1 + f_2$

#### Step 1

Find $\theta_0, \theta_1, \theta_2$, where $\theta_{i}^2 = |f_{j}|^2$.

In [20]:
eqs = [
    sp.Eq(-2*(xs[0]**1 + xs[1]**1 + xs[2]**1), ys[0]),
    sp.Eq(+2*(xs[0]**2 + xs[1]**2 + xs[2]**2), ys[1]),
    sp.Eq(-2*(xs[0]**3 + xs[1]**3 + xs[2]**3), ys[2]),
]

p = eliminate(eqs, xs)
p

48*x_0**3 + 24*x_0**2*y_0 + x_0*(6*y_0**2 - 12*y_1) + y_0**3 - 6*y_0*y_1 + 8*y_2

In [21]:
sp.Poly(p, xs[0]).coeffs()

[48, 24*y_0, 6*y_0**2 - 12*y_1, y_0**3 - 6*y_0*y_1 + 8*y_2]

In [22]:
def find_theta2s(f):
    f1 = f
    f2 = f1 @ f1
    f4 = f2 @ f2
    f6 = f4 @ f2
    y0 = np.trace(f2)
    y1 = np.trace(f4)
    y2 = np.trace(f6)
    p = np.poly1d([48, 24*y0, 6*(y0**2 - 2*y1), y0**3 - 6*y0*y1 + 8*y2])
    return p.roots

In [23]:
_theta0 = 1
_theta1 = 2
_theta2 = 3

_f = np.array(_theta0 * g6.e01 + _theta1 * g6.e23 + _theta2 * g6.e45).astype(np.float32)

__theta0, __theta1, __theta2 = np.sqrt(find_theta2s(_f))

_f, __theta0, __theta1, __theta2

(array([[ 0.,  1.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  2.,  0.,  0.],
        [ 0.,  0., -2.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  3.],
        [ 0.,  0.,  0.,  0., -3.,  0.]], dtype=float32),
 3.0000000000000013,
 1.9999999999999987,
 0.9999999999999999)

#### Step 2

Express $f_0, f_1, f_2$ in terms of $f$ and $\theta_0, \theta_1, , \theta_2$ by solving the following system (see [1] for derivation):

${\begin{bmatrix}1&1&1\\\theta_0^2&\theta_1^2&\theta_2^2\\\theta_0^4&\theta_1^4&\theta_2^4\\\end{bmatrix}}\,{\begin{bmatrix}f_0\\f_1\\f_2\\\end{bmatrix}}={\begin{bmatrix}f\\-f^3\\f^5\\\end{bmatrix}}$

In [24]:
T = sp.Matrix([
    [thetas[0]**0,thetas[1]**0,thetas[2]**0],
    [thetas[0]**2,thetas[1]**2,thetas[2]**2],
    [thetas[0]**4,thetas[1]**4,thetas[2]**4]
])
_f = sp.Matrix([[f, -f**3, f**5]])
_fs = sp.Matrix([fs[:3]])

eq0 = sp.Eq(T @ _fs.T, _f.T)
eq0

Eq(Matrix([
[                                    f_0 + f_1 + f_2],
[\theta_0**2*f_0 + \theta_1**2*f_1 + \theta_2**2*f_2],
[\theta_0**4*f_0 + \theta_1**4*f_1 + \theta_2**4*f_2]]), Matrix([
[    f],
[-f**3],
[ f**5]]))

Solve for $f_0, f_1, f_2$.

In [25]:
eq1 = sp.Eq(_fs.T, 1/T.det() * (T.adjugate() @ _f.T))
sp.simplify(eq1)

Eq(Matrix([
[f_0],
[f_1],
[f_2]]), Matrix([
[ f*(\theta_1**2*\theta_2**2 + \theta_1**2*f**2 + \theta_2**2*f**2 + f**4)/(\theta_0**4 - \theta_0**2*\theta_1**2 - \theta_0**2*\theta_2**2 + \theta_1**2*\theta_2**2)],
[-f*(\theta_0**2*\theta_2**2 + \theta_0**2*f**2 + \theta_2**2*f**2 + f**4)/(\theta_0**2*\theta_1**2 - \theta_0**2*\theta_2**2 - \theta_1**4 + \theta_1**2*\theta_2**2)],
[ f*(\theta_0**2*\theta_1**2 + \theta_0**2*f**2 + \theta_1**2*f**2 + f**4)/(\theta_0**2*\theta_1**2 - \theta_0**2*\theta_2**2 - \theta_1**2*\theta_2**2 + \theta_2**4)]]))

In [26]:
def decompose_antisymmetric(f):
    f1 = f
    f2 = f1 @ f1
    f3 = f2 @ f1
    f4 = f3 @ f1
    f5 = f4 @ f1
    f6 = f5 @ f1
    y0 = np.trace(f2)
    y1 = np.trace(f4)
    y2 = np.trace(f6)
    t = np.poly1d([48, 24*y0, 6*(y0**2 - 2*y1), y0**3 - 6*y0*y1 + 8*y2]).roots
    dt = (t[0] * t[1]**2 - t[0]**2 * t[1]) + (t[0]**2 * t[2] - t[0] * t[2]**2) + (t[1]*t[2]**2 - t[1]**2 * t[2]) 
    if np.isclose(dt, 0):
        raise NotImplementedError
    f0 = ((t[2] - t[1])*f5 + (t[2]**2 - t[1]**2)*f3 + (t[1]*t[2]**2 - t[1]**2*t[2])*f)/dt
    f1 = ((t[0] - t[2])*f5 + (t[0]**2 - t[2]**2)*f3 + (t[2]*t[0]**2 - t[2]**2*t[0])*f)/dt
    f2 = ((t[1] - t[0])*f5 + (t[1]**2 - t[0]**2)*f3 + (t[0]*t[1]**2 - t[0]**2*t[1])*f)/dt
    return f0, f1, f2

In [27]:
_theta0 = 1
_theta1 = 2
_theta2 = 3

_f = np.array(_theta0 * g6.e01 + _theta1 * g6.e23 + _theta2 * g6.e45).astype(np.float32)

_f0, _f1, _f2 = decompose_antisymmetric(_f)

#print(_f, _f0, _f1, _f2)

np.allclose(_f, _f0 + _f1 + _f2), is_ortho(_f0, _f1), is_ortho(_f0, _f2), is_ortho(_f1, _f2)

(True, True, True, True)

### Orthogonal Decomposition of Antisymmetric Matrix

In [28]:
def generate_polynomial(n):
    m = n//2
    eqs = [sp.Eq((-1)**(i+1) * 2 * sum(xs[j]**(i+1) for j in range(m)), ys[i]) for i in range(m)]
    return eliminate(eqs, xs)

In [29]:
p = generate_polynomial(8)
p

384*x_0**4 + 192*x_0**3*y_0 + x_0**2*(48*y_0**2 - 96*y_1) + x_0*(8*y_0**3 - 48*y_0*y_1 + 64*y_2) + y_0**4 - 12*y_0**2*y_1 + 32*y_0*y_2 + 12*y_1**2 - 48*y_3

In [30]:
sp.Poly(p, xs[0]).coeffs()

[384,
 192*y_0,
 48*y_0**2 - 96*y_1,
 8*y_0**3 - 48*y_0*y_1 + 64*y_2,
 y_0**4 - 12*y_0**2*y_1 + 32*y_0*y_2 + 12*y_1**2 - 48*y_3]

In [39]:
def generate_equation(n):
    m = n//2
    T = sp.Matrix([[thetas[j]**(2*i) for j in range(m)] for i in range(m)])
    _f = sp.Matrix([[(-1)**(i) * f**(2*i+1) for i in range(m)]])
    _fs = sp.Matrix([fs[:m]])
    return T, _f, _fs

In [40]:
T, _f, _fs = generate_equation(8)

In [41]:
eq0 = sp.Eq(T @ _fs.T, _f.T)
eq0

Eq(Matrix([
[                                                f_0 + f_1 + f_2 + f_3],
[\theta_0**2*f_0 + \theta_1**2*f_1 + \theta_2**2*f_2 + \theta_3**2*f_3],
[\theta_0**4*f_0 + \theta_1**4*f_1 + \theta_2**4*f_2 + \theta_3**4*f_3],
[\theta_0**6*f_0 + \theta_1**6*f_1 + \theta_2**6*f_2 + \theta_3**6*f_3]]), Matrix([
[    f],
[-f**3],
[ f**5],
[-f**7]]))

In [42]:
eq1 = sp.Eq(_fs.T, 1/T.det() * (T.adjugate() @ _f.T))
sp.simplify(eq1)

Eq(Matrix([
[f_0],
[f_1],
[f_2],
[f_3]]), Matrix([
[-f*(\theta_1**2*\theta_2**2*\theta_3**2 + \theta_1**2*\theta_2**2*f**2 + \theta_1**2*\theta_3**2*f**2 + \theta_1**2*f**4 + \theta_2**2*\theta_3**2*f**2 + \theta_2**2*f**4 + \theta_3**2*f**4 + f**6)/(\theta_0**6 - \theta_0**4*\theta_1**2 - \theta_0**4*\theta_2**2 - \theta_0**4*\theta_3**2 + \theta_0**2*\theta_1**2*\theta_2**2 + \theta_0**2*\theta_1**2*\theta_3**2 + \theta_0**2*\theta_2**2*\theta_3**2 - \theta_1**2*\theta_2**2*\theta_3**2)],
[ f*(\theta_0**2*\theta_2**2*\theta_3**2 + \theta_0**2*\theta_2**2*f**2 + \theta_0**2*\theta_3**2*f**2 + \theta_0**2*f**4 + \theta_2**2*\theta_3**2*f**2 + \theta_2**2*f**4 + \theta_3**2*f**4 + f**6)/(\theta_0**2*\theta_1**4 - \theta_0**2*\theta_1**2*\theta_2**2 - \theta_0**2*\theta_1**2*\theta_3**2 + \theta_0**2*\theta_2**2*\theta_3**2 - \theta_1**6 + \theta_1**4*\theta_2**2 + \theta_1**4*\theta_3**2 - \theta_1**2*\theta_2**2*\theta_3**2)],
[-f*(\theta_0**2*\theta_1**2*\theta_3**2 + \theta_0**2*\the

In [43]:
sp.simplify(T.det())

\theta_0**6*\theta_1**4*\theta_2**2 - \theta_0**6*\theta_1**4*\theta_3**2 - \theta_0**6*\theta_1**2*\theta_2**4 + \theta_0**6*\theta_1**2*\theta_3**4 + \theta_0**6*\theta_2**4*\theta_3**2 - \theta_0**6*\theta_2**2*\theta_3**4 - \theta_0**4*\theta_1**6*\theta_2**2 + \theta_0**4*\theta_1**6*\theta_3**2 + \theta_0**4*\theta_1**2*\theta_2**6 - \theta_0**4*\theta_1**2*\theta_3**6 - \theta_0**4*\theta_2**6*\theta_3**2 + \theta_0**4*\theta_2**2*\theta_3**6 + \theta_0**2*\theta_1**6*\theta_2**4 - \theta_0**2*\theta_1**6*\theta_3**4 - \theta_0**2*\theta_1**4*\theta_2**6 + \theta_0**2*\theta_1**4*\theta_3**6 + \theta_0**2*\theta_2**6*\theta_3**4 - \theta_0**2*\theta_2**4*\theta_3**6 - \theta_1**6*\theta_2**4*\theta_3**2 + \theta_1**6*\theta_2**2*\theta_3**4 + \theta_1**4*\theta_2**6*\theta_3**2 - \theta_1**4*\theta_2**2*\theta_3**6 - \theta_1**2*\theta_2**6*\theta_3**4 + \theta_1**2*\theta_2**4*\theta_3**6

In [45]:
sp.expand(sp.simplify(T.adjugate() @ _f.T))

Matrix([
[-\theta_1**6*\theta_2**4*\theta_3**2*f - \theta_1**6*\theta_2**4*f**3 + \theta_1**6*\theta_2**2*\theta_3**4*f - \theta_1**6*\theta_2**2*f**5 + \theta_1**6*\theta_3**4*f**3 + \theta_1**6*\theta_3**2*f**5 + \theta_1**4*\theta_2**6*\theta_3**2*f + \theta_1**4*\theta_2**6*f**3 - \theta_1**4*\theta_2**2*\theta_3**6*f - \theta_1**4*\theta_2**2*f**7 - \theta_1**4*\theta_3**6*f**3 + \theta_1**4*\theta_3**2*f**7 - \theta_1**2*\theta_2**6*\theta_3**4*f + \theta_1**2*\theta_2**6*f**5 + \theta_1**2*\theta_2**4*\theta_3**6*f + \theta_1**2*\theta_2**4*f**7 - \theta_1**2*\theta_3**6*f**5 - \theta_1**2*\theta_3**4*f**7 - \theta_2**6*\theta_3**4*f**3 - \theta_2**6*\theta_3**2*f**5 + \theta_2**4*\theta_3**6*f**3 - \theta_2**4*\theta_3**2*f**7 + \theta_2**2*\theta_3**6*f**5 + \theta_2**2*\theta_3**4*f**7],
[ \theta_0**6*\theta_2**4*\theta_3**2*f + \theta_0**6*\theta_2**4*f**3 - \theta_0**6*\theta_2**2*\theta_3**4*f + \theta_0**6*\theta_2**2*f**5 - \theta_0**6*\theta_3**4*f**3 - \theta_0**6*\the